In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Perguntas e respostas com modelos genertivos na Vertex AI

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Execute no Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      Veja no GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Execute no Vertex AI Workbench
    </a>
  </td>
</table>

## Visão geral

Grandes modelos de linguagem podem ser usados para várias tarefas de processamento de linguagem natural, incluindo perguntas e respostas (Q&A). Esses modelos são treinados em uma grande quantidade de dados de texto e podem gerar respostas de alta qualidade para uma ampla gama de perguntas. Uma coisa a observar aqui é que a maioria dos modelos tem datas limite em relação ao seu conhecimento, e perguntar qualquer coisa muito recente pode resultar em uma resposta incompleta, imaginativa ou incorreta (ou seja, uma alucinação).

Este bloco de anotações aborda os fundamentos dos prompts para responder a perguntas usando um modelo generativo. Além disso, apresenta o `domínio aberto` (conhecimento disponível na internet pública) e o `domínio fechado` (conhecimento que é mais privado - normalmente conhecimento empresarial ou pessoal).

Saiba mais sobre o design de prompt na [documentação oficial](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview#prompt_structure).

### Objetivo

Ao final deste notebook, você será capaz de escrever prompts para os seguintes cenários:

* Perguntas de **domínio aberto**:
     * Solicitação one-shot
     * Solicitação few-shot


* Perguntas de **domínio fechado**:
     * Fornecer conhecimento personalizado como contexto
     * Instrução de ajuste das saídas
     * Solicitação few-shot

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Generative AI Studio

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

## Primeiros Passos

### Instalando os SDK da Vertex AI e da Cloud Translate API

In [ ]:
!pip install google-cloud-aiplatform google-cloud-translate --upgrade --user

**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [ ]:
# # Reinicia automaticamente o kernel após as instalações para que seu ambiente possa acessar os novos pacotes
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../setup-env/README.md).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [ ]:
# import vertexai

# PROJECT_ID = "[seu-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import pandas as pd
from vertexai.preview.language_models import TextGenerationModel

#### Carregando o modelo `text-bison`

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

#### Criando a função *wrapper* para utilizar os modelos em Português

Até o momento desde treinamento, as API do Generative AI Studio suportam somente interações no idioma inglês. Para fazermos interações utilizando o idioma português, vamos utilizar a [Cloud Translation API](https://cloud.google.com/translate) para traduzir as nossas solicitações do português para o inglês e as respostas da API de inglês para português.

In [ ]:
from google.cloud import translate

project_id = !gcloud config list project
project_id = project_id[1].split('=')[1].strip()
parent = f'projects/' + project_id


def traduza(texto, idioma_destino):
    client = translate.TranslationServiceClient()

    response = client.translate_text(
        parent=parent,
        contents=[texto],
        target_language_code=idioma_destino,
        mime_type="text/plain"
    )

    return response.translations[0].translated_text

## Perguntas e respostas

Os recursos de resposta a perguntas exigem o fornecimento de um prompt ou uma pergunta que o modelo possa usar para gerar uma resposta. O prompt pode ser algumas palavras ou algumas frases completas, dependendo da complexidade da pergunta.

Ao criar um prompt de resposta a perguntas, é essencial ser específico e fornecer o máximo de contexto possível. Isso ajuda o modelo a entender a intenção por trás da pergunta e gerar uma resposta relevante. Por exemplo, se você quiser perguntar:

```
"Qual é a capital da França?",

então um bom prompt poderia ser:

"Diga-me o nome da cidade que é capital da França."

```

Além de ser específico, o prompt também deve ser gramaticalmente correto e livre de erros ortográficos. Isso ajuda o modelo a gerar uma resposta fácil de entender e contém menos erros ou imprecisões.

Ao fornecer prompts específicos e ricos em contexto, você pode ajudar o modelo a entender a intenção por trás da pergunta e gerar respostas precisas e relevantes.

Abaixo estão algumas diferenças entre as categorias **domínio aberto** e **domínio fechado** para prompts de resposta a perguntas.

* **Domínio aberto**: Todas as perguntas cujas respostas já estão disponíveis online. Eles podem pertencer a qualquer categoria, como história, geografia, países, política, química, etc. Isso inclui perguntas triviais ou de conhecimento geral, como:

```
P: Quem ganhou o ouro olímpico na natação?
P: Quem é o presidente de [dado país]?
P: Quem escreveu [livro específico]"?
```

Lembre-se do corte de treinamento de modelos generativos, pois perguntas envolvendo informações mais recentes do que aquelas com as quais o modelo foi treinado podem fornecer respostas incorretas ou imaginativas.


* **Domínio fechado**: Se você possui alguma base de conhecimento interna não disponível na internet pública, então ela pertence à categoria _domínio fechado_.
Você pode passar esse conhecimento "privado" como contexto para o modelo. Se solicitado corretamente, é mais provável que o modelo responda dentro do contexto fornecido e menos provável que dê respostas além da Internet aberta.

Considere o exemplo de criar um bot de perguntas e respostas sobre a documentação interna do produto. Nesse caso, você pode passar a documentação completa para o modelo e solicitar que ele responda apenas com base nisso.

Prompt típico para **domínio fechado**:

```
Prompt: f""" Resposta do contexto abaixo: \n\n
contexto: {sua base de conhecimento} \n
pergunta: {pergunta específica para essa base de conhecimento} \n
resposta: {a ser previsto pelo modelo} \n
"""
```

Abaixo estão alguns exemplos para entender esses diferentes tipos de prompts.

### Domínio aberto

#### Prompt zero-shot

In [ ]:
prompt = traduza("""Pregunta: ¿Quién fue el presidente de Argentina en 2010?\n
                    Respuesta:
                     """, "en")
print(
    traduza(generation_model.predict(prompt, max_output_tokens=256, temperature=0.1).text, "es")
)

In [ ]:
prompt = traduza("""Pregunta: ¿Cuál es la montaña más alta del mundo?\n
                     Respuesta:
         """, "en")
print(
    traduza(generation_model.predict(prompt, max_output_tokens=20, temperature=0.1).text, "es")
)

#### Prompt few-shot

Digamos que você queira obter uma resposta curta do modelo (como apenas um nome específico). Para fazer isso, você pode aproveitar um prompt few-shot e fornecer exemplos ao modelo para ilustrar o comportamento esperado.

In [ ]:
prompt = traduza("""Pregunta: Quem é atualmente o presidente da França?\n
                Respuesta: Emmanuel Macron \n\n

                Pregunta: Quem inventou o telefone? \n
                Respuesta: Alexander Graham Bell \n\n

                Pregunta: Quem escreveu o livro "1984"?
                Respuesta: George Orwell

                Pregunta: Quem descobriu a penicilina?
                Respuesta:
                """, "en")
print(
    traduza(generation_model.predict(prompt, max_output_tokens=20, temperature=0.1).text, "es")
)

#### Prompt zero-shot vs few-shot

O prompt zero-shot pode ser útil para gerar texto rapidamente para novas tarefas, mas a qualidade do texto gerado pode ser inferior à de um prompt few-shot com exemplos bem escolhidos. O prompt few-shot geralmente é mais adequado para tarefas que exigem um alto grau de especificidade ou conhecimento específico do domínio, mas requer algum pensamento adicional e potencialmente dados para configurar o prompt.

### Domínio fechado

#### Adicionando conhecimento como contexto nos prompts

Imagine um cenário em que você gostaria de criar um bot de perguntas e respostas que aceita a documentação interna e permite que os usuários façam perguntas sobre ela.

No exemplo abaixo, o Google Cloud Storage e a documentação da política de conteúdo são adicionados ao prompt, para que a API PaLM possa usá-lo para responder a perguntas subsequentes com o contexto fornecido.

In [ ]:
context = traduza("""
Política de almacenamiento y contenido \n
¿Qué tan duraderos son mis datos en Cloud Storage? \norte
El almacenamiento en la nube está diseñado para una durabilidad anual del 99,999999999 % (11 9), que es adecuado incluso para el almacenamiento principal y
aplicaciones críticas para el negocio. Este alto nivel de durabilidad se logra mediante la codificación de borrado que almacena fragmentos de datos de forma redundante.
en múltiples dispositivos ubicados en múltiples zonas de disponibilidad.
Los objetos escritos en Cloud Storage deben almacenarse de forma redundante en al menos dos zonas de disponibilidad diferentes antes de
la escritura se reconoce como exitosa. Las sumas de verificación se almacenan y revalidan regularmente para verificar proactivamente que los datos
integridad de todos los datos en reposo, así como la detección de corrupción de datos en tránsito. Si es necesario, las correcciones se realizan automáticamente.
hecho usando datos redundantes. Los clientes pueden habilitar opcionalmente el control de versiones de objetos para agregar protección contra la eliminación accidental.
""", "en")

question = traduza("¿Cómo podemos lograr una alta disponibilidad?", "en")

prompt = traduza(f"""Responda la siguiente pregunta dado el contexto a continuación:
Contexto: {context} \n
Pregunta: {question} \n
Respuesta:
""", "en")

print(
    traduza(generation_model.predict(prompt).text, "es")
)

#### Ajustando as instruções de output do modelo

Outra maneira de ajudar os modelos de linguagem é fornecer instruções adicionais para enquadrar a saída no prompt. Para garantir que o modelo não responda a nada fora do contexto, o prompt pode especificar que a resposta deve ser "Informações não disponíveis no contexto fornecido", se for o caso.

In [ ]:
question = traduza("¿Qué tipo de máquinas se requieren para alojar modelos en Vertex AI?", "en")
prompt = traduza(f"""Responda la siguiente pregunta usando la información disponible como {{Contexto:}}. \n
Si la respuesta no está disponible en {{Contexto:}} y no está seguro del resultado, por favor
diga "Información no disponible en el contexto proporcionado". \n\n""", "en")
prompt += traduza(f"Contexto: {context} \n", "en")
prompt += traduza(f"Pregunta: {question} \n", "en")
prompt += traduza("Respuesta: ", "en")

print("[Respuesta]")
print(
    traduza(generation_model.predict(prompt, max_output_tokens=256, temperature=0.3).text, "es")
)

#### Prompt few-shot

In [ ]:
prompt = traduza("""
Contexto:
El término "inteligencia artificial" fue acuñado por primera vez por John McCarthy en 1956. Desde entonces, la IA se ha convertido en un vasto
campo con innumerables aplicaciones, desde autos sin conductor hasta asistentes virtuales como Siri y Alexa.

Pregunta:
¿Qué es la inteligencia artificial?

Responder:
La inteligencia artificial se refiere a la simulación de la inteligencia humana en máquinas programadas para pensar y aprender como humanos.

---

Contexto:
Los hermanos Wright, Orville y Wilbur, fueron dos pioneros de la aviación estadounidense a quienes se les atribuye la invención y
construir el primer avión exitoso del mundo y hacer el primer vuelo más pesado que el aire controlado, propulsado y sostenido por humanos,
   el 17 de diciembre de 1903.

Pregunta:
¿Quiénes eran los hermanos Wright?

Responder:
Los hermanos Wright fueron pioneros de la aviación estadounidense que inventaron y construyeron el primer avión exitoso del mundo.
y realizó el primer vuelo humano controlado, propulsado y sostenido más pesado que el aire el 17 de diciembre de 1903.

---

Contexto:
La Mona Lisa es un retrato del siglo XVI pintado por Leonardo da Vinci durante el Renacimiento italiano. Es uno de
las pinturas más famosas del mundo, conocidas por la enigmática sonrisa de la mujer representada en la pintura.

Pregunta:
¿Quién pintó la Mona Lisa?

Responder:
""", "en")
print(traduza(generation_model.predict(prompt,).text, "es"))

### Perguntas e resposta extrativas

No próximo exemplo, o modelo generativo é guiado para entender o significado da pergunta e da passagem e identificar as informações relevantes na passagem que responde à pergunta. O modelo recebe uma pergunta e uma passagem de texto e é solicitado a encontrar a resposta para a pergunta dentro da passagem. A resposta é tipicamente uma frase ou sentença.

In [ ]:
prompt = traduza("""
Antecedentes: Hay evidencia de que ha habido cambios significativos en la vegetación de la selva amazónica durante los últimos 21.000 años a través del Último Máximo Glacial (LGM) y la subsiguiente deglaciación.
Los análisis de los depósitos de sedimentos de paleolagos de la cuenca del Amazonas y el abanico del Amazonas indican que la precipitación en la cuenca durante el LGM fue menor que en la actualidad, y esto fue casi seguro
asociado a la reducida cobertura de vegetación tropical húmeda en la cuenca. Sin embargo, existe un debate sobre cuán extensa fue esta reducción. Algunos científicos argumentan que la selva tropical se ha reducido a pequeños
refugios aislados separados por bosques abiertos y pastizales; otros científicos argumentan que la selva tropical permaneció prácticamente intacta, pero se extendió menos hacia el norte, el sur y el este de lo que se ve hoy.
Este debate ha resultado difícil de resolver porque las limitaciones prácticas de trabajar en la selva tropical significan que el muestreo de datos se desvía del centro de la cuenca del Amazonas, y ambos
Las explicaciones están razonablemente bien respaldadas por los datos disponibles.

P: ¿Qué significa LGM?
A: Último Máximo Glacial.

P: ¿Qué indica el análisis de los depósitos de sedimentos?
R: La precipitación en la cuenca durante el LGM fue menor que en la actualidad.

P: ¿Cuáles son algunos de los argumentos de los científicos?
R: La selva tropical se ha reducido a pequeños refugios aislados separados por bosques abiertos y pastizales.

P: ¿Ha habido grandes cambios en la vegetación de la selva amazónica en los últimos cuántos años?
R: 21.000.

P: ¿Qué causó los cambios en la vegetación de la selva amazónica?
A: El Último Máximo Glacial (LGM) y posterior desglaciación

P: ¿Qué se analizó para comparar las lluvias amazónicas del pasado y del presente?
A: Depósitos de sedimentos.

P: ¿Qué se atribuyó a la menor precipitación en la Amazonía durante la LGM?
A:
""", "en")

print(traduza(generation_model.predict(prompt).text, "es"))

### Avaliação de respostas

Você pode avaliar os resultados da tarefa de pergunta e resposta se as respostas de verdade de cada pergunta estiverem disponíveis. No prompt zero-shot, você só pode usar perguntas de 'domínio aberto'. No entanto, com perguntas de 'domínio fechado', você pode adicionar contexto e avaliar de forma semelhante. Para mostrar como isso funciona, comece criando um dataframe simples com perguntas e respostas de verdade.

In [ ]:
qa_data = {
    "pregunta": [
         "En la barra de direcciones de los navegadores, ¿qué significa 'www'?",
         "¿Quién fue la primera mujer en ganar un premio Nobel?",
         "¿Cuál es el nombre del océano más grande de la Tierra?",
     ],
     "answer_groundtruth": ["World Wide Web", "Marie Curie", "El Océano Pacífico"],
}
qa_data_df = pd.DataFrame(qa_data)
qa_data_df

Agora que você tem os dados com perguntas e respostas básicas, pode chamar o modelo de geração PaLM 2 para cada linha de revisão usando a função `apply`. Cada linha usará o prompt dinâmico para prever a resposta usando a API PaLM. Vamos salvar os resultados na coluna `answer_prediction`.

In [ ]:
def get_answer(row):
    prompt = traduza(f"""Responda las siguientes preguntas con la mayor precisión posible.\n\n
                     pregunta: {traduza(row, "en")}
                     respuesta:
                     """, "en")
    return traduza(generation_model.predict(prompt=prompt).text, "es")

qa_data_df["answer_prediction"] = qa_data_df["pregunta"].apply(get_answer)
qa_data_df

Você pode querer avaliar as respostas previstas pela API do PaLM. No entanto, será mais complexo do que a classificação do texto, pois as respostas podem diferir da verdade e podem ser apresentadas em um pouco mais/menos palavras.

Por exemplo, você pode observar a pergunta "Qual é o nome do maior oceano da Terra?" e veja que o modelo previu "Oceano Pacífico" quando um rótulo de verdade é "O Oceano Pacífico" com o extra "O". Agora, se você usar as métricas de classificação simples, considerará isso uma previsão errada, pois as strings originais e previstas têm uma diferença. No entanto, você pode ver que a resposta está correta, pois um "The" extra está causando o problema. É um problema simples de comparação de strings.

A solução para a comparação de strings em que `ground_thruth` e `predicted` podem ter algumas letras extras ou menos, uma abordagem é usar um algoritmo de correspondência difusa.
A correspondência de string difusa usa [Distância Levenshtein](https://en.wikipedia.org/wiki/Levenshtein_distance) para calcular as diferenças entre duas strings.

Aqui um exemplo usando a biblioteca `fuzzywuzzy`, que nos dá a `distância de Levenshtein` entre duas strings, mas em proporção. A pontuação bruta de proporção mede a similaridade da string como um int no intervalo [0, 100]. Para duas strings X e Y, a pontuação é definida por `int(round((2.0 * M / T) * 100))` onde `T` é o número total de caracteres em ambas as strings e `M` é o número de correspondências nas duas strings.

Leia mais aqui sobre a [fórmula de proporção](https://anhaidgroup.github.io/py_stringmatching/v0.3.x/Ratio.html):

Você pode ver um exemplo para entender melhor.
```
String1: "isto é um teste"
String2: "isto é um teste!"

Razão Fuzz => 97 #

Fuzz Partial Ratio => 100 #Como a maioria dos caracteres são iguais e em uma sequência semelhante, o algoritmo calcula a proporção parcial como 100 e ignora adições simples (novos caracteres).
```

Primeiro instale os pacotes `fuzzywuzzy` e `python-Levenshtein`:

In [ ]:
!pip install -q python-Levenshtein --upgrade --user
!pip install -q fuzzywuzzy --upgrade --user

E então calcule o match fuzzy:

In [ ]:
from fuzzywuzzy import fuzz


def get_fuzzy_match(df):
    return fuzz.partial_ratio(df["answer_groundtruth"], df["answer_prediction"])


qa_data_df["match_score"] = qa_data_df.apply(get_fuzzy_match, axis=1)
qa_data_df

Agora que você tem a pontuação de correspondência individual (parcial), pode obter a média ou a média de toda a coluna para ter uma noção dos dados gerais.

Pontuações próximas a 100 significam que o PaLM 2 pode prever com mais precisão; se a pontuação estiver próxima de 50 ou 0, não teve um bom desempenho.

In [ ]:
print(
    "En promedio, la puntuación de todas las predicciones de PaLM 2 fue: ",
    qa_data_df["match_score"].mean(),
    " %",
)

Nesse caso, você obteve um score de mais ou menos 97% como pontuação média, pois algumas previsões faltem algumas palavras. Isso significa que você está muito próximo da verdade básica e algumas respostas estão apenas perdendo a escrita exata da verdade básica.